In [113]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import pickle
import warnings
from datetime import timedelta 
import datetime

from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from statsmodels.tsa.arima.model import ARIMA
import warnings
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric
from sklearn.metrics import mean_squared_error
import pmdarima
from sklearn.ensemble import RandomForestRegressor

# -1 Loading Data

In [40]:
#csv_file_path = 
train_df = pd.read_csv('../raw_data/sales_train_validation.csv')
prices_df = pd.read_csv('../raw_data/sell_prices.csv')
calendar_df = pd.read_csv('../raw_data/calendar.csv')

In [41]:
train_df_filtered = train_df.iloc[0:200,:]

In [42]:
train_df_sample = train_df_filtered.melt(id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'])

In [43]:
train_df_sample.rename(columns={'variable': 'd', 'value':'sales'},inplace=True)

In [44]:
merge_df = train_df_sample.merge(calendar_df,on='d',how='left')

In [45]:
merge_df = merge_df.merge(prices_df,on=['store_id', 'item_id','wm_yr_wk'],how='left')
merge_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382595,HOBBIES_1_202_CA_1_validation,HOBBIES_1_202,HOBBIES_1,HOBBIES,CA_1,CA,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,1.77
382596,HOBBIES_1_203_CA_1_validation,HOBBIES_1_203,HOBBIES_1,HOBBIES,CA_1,CA,d_1913,1,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,12.98
382597,HOBBIES_1_204_CA_1_validation,HOBBIES_1_204,HOBBIES_1,HOBBIES,CA_1,CA,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,9.72
382598,HOBBIES_1_205_CA_1_validation,HOBBIES_1_205,HOBBIES_1,HOBBIES,CA_1,CA,d_1913,0,2016-04-24,11613,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,5.88


# 0 Importing Data

In [46]:
# Load your dataset
merge_df['date'] = pd.to_datetime(merge_df['date'])
merge_df.set_index('date', inplace=True)

merge_df
# 382600 rows × 64 columns

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,wm_yr_wk,weekday,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
date,,,,,,,,,,,,,,,,,,,,,
2011-01-29,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,Saturday,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2011-01-29,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,Saturday,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2011-01-29,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,Saturday,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2011-01-29,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,Saturday,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2011-01-29,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,11101,Saturday,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-04-24,HOBBIES_1_202_CA_1_validation,HOBBIES_1_202,HOBBIES_1,HOBBIES,CA_1,CA,d_1913,0,11613,Sunday,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,1.77
2016-04-24,HOBBIES_1_203_CA_1_validation,HOBBIES_1_203,HOBBIES_1,HOBBIES,CA_1,CA,d_1913,1,11613,Sunday,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,12.98
2016-04-24,HOBBIES_1_204_CA_1_validation,HOBBIES_1_204,HOBBIES_1,HOBBIES,CA_1,CA,d_1913,0,11613,Sunday,...,4,2016,NaN,NaN,NaN,NaN,0,0,0,9.72


In [47]:
#FILLING THE EMPTY PLACES
merge_df['sell_price'].fillna(0, inplace=True)
merge_df['event_name_1'].fillna('missing', inplace=True)
merge_df['event_type_1'].fillna('missing', inplace=True)
merge_df['event_name_2'].fillna('missing', inplace=True)
merge_df['event_type_2'].fillna('missing', inplace=True)


/tmp/ipykernel_4342/1627116259.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merge_df['sell_price'].fillna(0, inplace=True)
/tmp/ipykernel_4342/1627116259.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [48]:
# Assuming df is your DataFrame with datetime index and "id" column
start_date = merge_df.index.min()
end_date = merge_df.index.max()

# Generate the complete date range
complete_date_range = pd.date_range(start=start_date, end=end_date)

# Get unique values of "id" column
unique_ids = merge_df['id'].unique()

merge_df = merge_df.reset_index()

# Create a MultiIndex with Cartesian product of date range and unique ids
multi_index = pd.MultiIndex.from_product([complete_date_range, unique_ids], names=['date', 'id'])

# Reindex the DataFrame using the MultiIndex
merge_df = merge_df.set_index(['date', 'id']).reindex(multi_index)

# Reset the index to make "date" and "id" columns again
merge_df = merge_df.reset_index()

In [49]:
merge_df.set_index("date",inplace=True)

In [50]:
# Scale 'sell_price' and 'year' by using MinMaxScaler
minmax_scaler = MinMaxScaler()

merge_df[['sell_price']] = minmax_scaler.fit_transform(merge_df[['sell_price']])
merge_df[['year']] = minmax_scaler.fit_transform(merge_df[['year']])

In [51]:
# Check unique values for 'cat_id'
print(f"The unique values for 'cat_id' are {merge_df['cat_id'].unique()}")

# Check unique values for 'store_id'
print(f"The unique values for 'store_id' are {merge_df['store_id'].unique()}")

# Instantiate the OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)

# Fit encoder for both 'cat_id' and 'store_id'
ohe.fit(merge_df[['cat_id', 'store_id']])

# Display the detected categories for both columns
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names for both columns
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the 'cat_id' and 'store_id' columns
encoded_columns = ohe.transform(merge_df[['cat_id', 'store_id']])

# Drop the original 'cat_id' and 'store_id' columns
merge_df.drop(columns=['cat_id', 'store_id'], inplace=True)

# Concatenate the encoded columns to the DataFrame
merge_df[ ohe.get_feature_names_out()] = encoded_columns


The unique values for 'cat_id' are ['HOBBIES']
The unique values for 'store_id' are ['CA_1']
The categories detected by the OneHotEncoder are [array(['HOBBIES'], dtype=object), array(['CA_1'], dtype=object)]
The column names for the encoded values are ['cat_id_HOBBIES' 'store_id_CA_1']


In [52]:
# Check unique values
print(f"The unique values for 'event_type_1' are {merge_df['event_type_1'].unique()}")

# Fit encoder
ohe.fit(merge_df[['event_type_1']])

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "cat_id" column
merge_df[ohe.get_feature_names_out()] = ohe.transform(merge_df[['event_type_1']])

# Drop the column "cat_id" which has been encoded
merge_df.drop(columns = ['event_type_1'], inplace = True)

The unique values for 'event_type_1' are ['missing' 'Sporting' 'Cultural' 'National' 'Religious']
The categories detected by the OneHotEncoder are [array(['Cultural', 'National', 'Religious', 'Sporting', 'missing'],
      dtype=object)]
The column names for the encoded values are ['event_type_1_Cultural' 'event_type_1_National' 'event_type_1_Religious'
 'event_type_1_Sporting' 'event_type_1_missing']


In [53]:
# Check unique values
print(f"The unique values for 'event_type_2' are {merge_df['event_type_2'].unique()}")

# Fit encoder
ohe.fit(merge_df[['event_type_2']])

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "cat_id" column
merge_df[ohe.get_feature_names_out()] = ohe.transform(merge_df[['event_type_2']])

# Drop the column "cat_id" which has been encoded
merge_df.drop(columns = ['event_type_2'], inplace = True)

The unique values for 'event_type_2' are ['missing' 'Cultural' 'Religious']
The categories detected by the OneHotEncoder are [array(['Cultural', 'Religious', 'missing'], dtype=object)]
The column names for the encoded values are ['event_type_2_Cultural' 'event_type_2_Religious' 'event_type_2_missing']


In [54]:
# Check unique values
print(f"The unique values for 'event_name_1' are {merge_df['event_name_1'].unique()}")

# Fit encoder
ohe.fit(merge_df[['event_name_1']])

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "cat_id" column
merge_df[ohe.get_feature_names_out()] = ohe.transform(merge_df[['event_name_1']])

# Drop the column "cat_id" which has been encoded
merge_df.drop(columns = ['event_name_1'], inplace = True)

The unique values for 'event_name_1' are ['missing' 'SuperBowl' 'ValentinesDay' 'PresidentsDay' 'LentStart'
 'LentWeek2' 'StPatricksDay' 'Purim End' 'OrthodoxEaster' 'Pesach End'
 'Cinco De Mayo' "Mother's day" 'MemorialDay' 'NBAFinalsStart'
 'NBAFinalsEnd' "Father's day" 'IndependenceDay' 'Ramadan starts'
 'Eid al-Fitr' 'LaborDay' 'ColumbusDay' 'Halloween' 'EidAlAdha'
 'VeteransDay' 'Thanksgiving' 'Christmas' 'Chanukah End' 'NewYear'
 'OrthodoxChristmas' 'MartinLutherKingDay' 'Easter']
The categories detected by the OneHotEncoder are [array(['Chanukah End', 'Christmas', 'Cinco De Mayo', 'ColumbusDay',
       'Easter', 'Eid al-Fitr', 'EidAlAdha', "Father's day", 'Halloween',
       'IndependenceDay', 'LaborDay', 'LentStart', 'LentWeek2',
       'MartinLutherKingDay', 'MemorialDay', "Mother's day",
       'NBAFinalsEnd', 'NBAFinalsStart', 'NewYear', 'OrthodoxChristmas',
       'OrthodoxEaster', 'Pesach End', 'PresidentsDay', 'Purim End',
       'Ramadan starts', 'StPatricksDay', 'SuperB

In [55]:
# Check unique values
print(f"The unique values for 'event_name_2' are {merge_df['event_name_2'].unique()}")

# Fit encoder
ohe.fit(merge_df[['event_name_2']])

# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")

# Display the generated names
print(f"The column names for the encoded values are {ohe.get_feature_names_out()}")

# Transform the current "cat_id" column
merge_df[ohe.get_feature_names_out()] = ohe.transform(merge_df[['event_name_2']])

# Drop the column "cat_id" which has been encoded
merge_df.drop(columns = ['event_name_2'], inplace = True)

The unique values for 'event_name_2' are ['missing' 'Easter' 'Cinco De Mayo' 'OrthodoxEaster' "Father's day"]
The categories detected by the OneHotEncoder are [array(['Cinco De Mayo', 'Easter', "Father's day", 'OrthodoxEaster',
       'missing'], dtype=object)]
The column names for the encoded values are ['event_name_2_Cinco De Mayo' 'event_name_2_Easter'
 "event_name_2_Father's day" 'event_name_2_OrthodoxEaster'
 'event_name_2_missing']


In [56]:
#Encoding Cyclical Features for weekdays
# Notice that Sat starts as 1 till Fri as 7 for 'wday'
merge_df['wday_sin'] = np.sin(2 * np.pi * merge_df['wday'] /7.0)
merge_df['wday_cos'] = np.cos(2 * np.pi * merge_df['wday'] /7.0)

In [57]:
#Encoding Cyclical Features for month

merge_df['month_sin'] = np.sin(2 * np.pi * merge_df['month'] /12.0)
merge_df['month_cos'] = np.cos(2 * np.pi * merge_df['month'] /12.0)

In [58]:
merge_df_scaled = merge_df.drop(columns=['d', 'wm_yr_wk','weekday'])

In [59]:
# Downcast numeric columns
numeric_columns = merge_df_scaled.select_dtypes(include=['int64', 'float64']).columns
merge_df_scaled[numeric_columns] = merge_df_scaled[numeric_columns].apply(lambda x: pd.to_numeric(x, downcast='integer' if np.issubdtype(x.dtype, np.integer) else 'float'))

# Confirm the new datatypes
print(merge_df_scaled.dtypes.unique())

[dtype('O') dtype('int8') dtype('float32')]


In [60]:
def feature_extraction(merge_df_scaled):
    # Ignore all warnings within this function
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    
        #lagged features
        for i in range(1, 8):
            merge_df_scaled[f'sales_lag_{i}'] = merge_df_scaled['sales'].shift(i)
    
        #lagged features per years
        for i in range(1, 4):
            merge_df_scaled[f'sales_lag_{i}years'] = merge_df_scaled['sales'].shift(i * 365)
    
            #rolling sum
        merge_df_scaled['rolling_sum_7'] = merge_df_scaled['sales'].rolling(window=7).sum()
        merge_df_scaled['rolling_sum_30'] = merge_df_scaled['sales'].rolling(window=30).sum()
        merge_df_scaled['rolling_sum_60'] = merge_df_scaled['sales'].rolling(window=60).sum()
        merge_df_scaled['rolling_sum_90'] = merge_df_scaled['sales'].rolling(window=90).sum()
        merge_df_scaled['rolling_sum_120'] = merge_df_scaled['sales'].rolling(window=120).sum()
    
        #rolling average
        merge_df_scaled['rolling_mean_7'] = merge_df_scaled['sales'].rolling(window=7).mean()
        merge_df_scaled['rolling_mean_30'] = merge_df_scaled['sales'].rolling(window=30).mean()
        merge_df_scaled['rolling_mean_60'] = merge_df_scaled['sales'].rolling(window=60).mean()
        merge_df_scaled['rolling_mean_90'] = merge_df_scaled['sales'].rolling(window=90).mean()
        merge_df_scaled['rolling_mean_120'] = merge_df_scaled['sales'].rolling(window=120).mean()
    
        #rolling stdv
        merge_df_scaled['rolling_stdv_7'] = merge_df_scaled['sales'].rolling(window=7).std()
        merge_df_scaled['rolling_stdv_30'] = merge_df_scaled['sales'].rolling(window=30).std()
        merge_df_scaled['rolling_stdv_60'] = merge_df_scaled['sales'].rolling(window=60).std()
        merge_df_scaled['rolling_stdv_90'] = merge_df_scaled['sales'].rolling(window=90).std()
        merge_df_scaled['rolling_stdv_120'] = merge_df_scaled['sales'].rolling(window=120).std()
    
        merge_df_scaled.fillna(0,inplace=True)
        

    return merge_df_scaled


In [61]:
def feature_extraction_transfer_test(train_df,test_df):

    # Ignore all warnings within this function
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
           
    
        #lagged features per years
        for i in range(1, 4):
            test_df[f'sales_lag_{i}years'] = train_df[f'sales_lag_{i}years'].iloc[-1]
    
            #rolling sum
        test_df['rolling_sum_7'] = train_df['rolling_sum_7'].iloc[-1]
        test_df['rolling_sum_30'] = train_df['rolling_sum_30'].iloc[-1]
        test_df['rolling_sum_60'] = train_df['rolling_sum_60'].iloc[-1]
        test_df['rolling_sum_90'] = train_df['rolling_sum_90'].iloc[-1]
        test_df['rolling_sum_120'] = train_df['rolling_sum_120'].iloc[-1]
        
        # Rolling average
        test_df['rolling_mean_7'] = train_df['rolling_mean_7'].iloc[-1]
        test_df['rolling_mean_30'] = train_df['rolling_mean_30'].iloc[-1]
        test_df['rolling_mean_60'] = train_df['rolling_mean_60'].iloc[-1]
        test_df['rolling_mean_90'] = train_df['rolling_mean_90'].iloc[-1]
        test_df['rolling_mean_120'] = train_df['rolling_mean_120'].iloc[-1]
        
        # Rolling standard deviation
        test_df['rolling_stdv_7'] = train_df['rolling_stdv_7'].iloc[-1]
        test_df['rolling_stdv_30'] = train_df['rolling_stdv_30'].iloc[-1]
        test_df['rolling_stdv_60'] = train_df['rolling_stdv_60'].iloc[-1]
        test_df['rolling_stdv_90'] = train_df['rolling_stdv_90'].iloc[-1]
        test_df['rolling_stdv_120'] = train_df['rolling_stdv_120'].iloc[-1]
    
        # Identify the last available date in the training data
        last_date_train = train_df.index[-1]
    
        # Fill in lagged features for the first few rows where future knowledge is available
        #for i in range(1, 8):
        #    # Identify the lagged date for the current lag
        #    lagged_date = last_date_train - pd.Timedelta(days=i)
            
            # Fill in the lagged sales values for corresponding lagged days from the training data
        #    test_df[f'sales_lag_{i}'] = test_df.index.map(lambda x: train_df.loc[x - pd.Timedelta(days=i), 'sales'] if x <= last_date_train else train_df[f'sales_lag_{i}'].iloc[-1])


        # Fill in lagged features for the first few rows where future knowledge is available
        for i in range(1, 8):
            test_df[f'sales_lag_{i}'] = np.nan  # Initialize with NaN
            
            # Iterate over each row in the test DataFrame
            for idx, row in test_df.iterrows():
                lagged_date = idx - pd.Timedelta(days=i)  # Calculate the lagged date
                
                # Check if the lagged date is within the training data range
                if lagged_date in train_df.index:
                    test_df.at[idx, f'sales_lag_{i}'] = train_df.loc[lagged_date, 'sales']
                else:
                    test_df.at[idx, f'sales_lag_{i}'] = train_df['sales'].iloc[-1]
    
    return test_df

In [62]:
def calc_rmsse(train, test, predictions):
    forecast_mse = mean_squared_error(test, predictions)
    train_mse = ((train - train.shift(1)) ** 2).mean()
    return np.sqrt(forecast_mse / train_mse)

In [63]:
def calculate_last_28_days_weights(df):
    most_recent_date = df.index.max()
    cutoff_date = most_recent_date - timedelta(days=27)
    last_28_days_df = df.loc[cutoff_date:most_recent_date]
    last_28_days_df['revenues'] = last_28_days_df['sales'] * last_28_days_df['sell_price']
    
    weights_df = last_28_days_df.groupby('id')['revenues'].sum()
    weights_df = pd.DataFrame(weights_df)
    
    total_revenues = last_28_days_df['revenues'].sum()
    weights_df['weights'] = weights_df['revenues'] / total_revenues
    
    return weights_df

In [77]:
def custom_ttsplit(product_data,num_splits=5, days_per_split=28):

    # Check if enough data is available
    if len(product_data) < days_per_split * num_splits:
        print(f"Not enough data for product {id} to perform {num_splits} splits with {days_per_split} days each.")
        
    
    # Initialize time_splits as a list of empty lists
    time_splits = [[] for _ in range(num_splits)]
    
    # Create data slices for 5-fold validation with each test slice being exactly 28 days
    for i in range(1, num_splits + 1):
        data_train = product_data.iloc[:-days_per_split * i]
        data_test = product_data.iloc[-days_per_split * i: (-days_per_split * (i - 1) if i > 1 else None)]   

        time_splits[i-1].append(data_train)
        time_splits[i-1].append(data_test)
        
        
    return time_splits
    



In [119]:
def custom_ttsplit_onlyfeatures(product_data,num_splits=5, days_per_split=28):

    # Check if enough data is available
    if len(product_data) < days_per_split * 2 * num_splits:
        print(f"Not enough data for product {id} to perform {num_splits} splits with {days_per_split} days each.")
        
    
    # Initialize time_splits as a list of empty lists
    time_splits = [[] for _ in range(num_splits)]
    
    # Create data slices for 5-fold validation with each test slice being exactly 28 days
    for i in range(1, num_splits + 1):
        data_train = product_data.iloc[:-days_per_split * i]
        data_test = product_data.iloc[-days_per_split * i: (-days_per_split * (i - 1) if i > 1 else None)]   

        data_train = feature_extraction(data_train)
        data_test = feature_extraction_transfer_test(data_train,data_test)

        data_test = data_test[data_train.columns]

        time_splits[i-1].append(data_train)
        time_splits[i-1].append(data_test)
        
        
    return time_splits
    



In [100]:
def custom_ttsplit_inclval(product_data,num_splits=5, days_per_split=28):

    # Check if enough data is available
    if len(product_data) < days_per_split * 2 * num_splits:
        print(f"Not enough data for product {id} to perform {num_splits} splits with {days_per_split} days each.")
        
    
    # Initialize time_splits as a list of empty lists
    time_splits = [[] for _ in range(num_splits)]
    
    # Create data slices for 5-fold validation with each test slice being exactly 28 days
    for i in range(1, num_splits + 1):
        data_train_incl_val = product_data.iloc[:-days_per_split * i]
        data_test = product_data.iloc[-days_per_split * i: (-days_per_split * (i - 1) if i > 1 else None)]   

        data_train_incl_val = feature_extraction(data_train_incl_val)
        data_test = feature_extraction_transfer_test(data_train_incl_val,data_test)
        data_train = data_train_incl_val[:-days_per_split]
        data_val = data_train_incl_val[-days_per_split:]

        time_splits[i-1].append(data_train)
        time_splits[i-1].append(data_val)
        time_splits[i-1].append(data_test)
        
        
    return time_splits
    



In [65]:
merge_df_scaled.drop(columns=["item_id","dept_id","state_id"],inplace=True)

In [66]:
merge_df_scaled.fillna(0,inplace=True)

In [67]:
merge_df_scaled["id"].unique()

array(['HOBBIES_1_001_CA_1_validation', 'HOBBIES_1_002_CA_1_validation',
       'HOBBIES_1_003_CA_1_validation', 'HOBBIES_1_004_CA_1_validation',
       'HOBBIES_1_005_CA_1_validation', 'HOBBIES_1_006_CA_1_validation',
       'HOBBIES_1_007_CA_1_validation', 'HOBBIES_1_008_CA_1_validation',
       'HOBBIES_1_009_CA_1_validation', 'HOBBIES_1_010_CA_1_validation',
       'HOBBIES_1_011_CA_1_validation', 'HOBBIES_1_012_CA_1_validation',
       'HOBBIES_1_013_CA_1_validation', 'HOBBIES_1_014_CA_1_validation',
       'HOBBIES_1_015_CA_1_validation', 'HOBBIES_1_016_CA_1_validation',
       'HOBBIES_1_017_CA_1_validation', 'HOBBIES_1_018_CA_1_validation',
       'HOBBIES_1_019_CA_1_validation', 'HOBBIES_1_020_CA_1_validation',
       'HOBBIES_1_021_CA_1_validation', 'HOBBIES_1_022_CA_1_validation',
       'HOBBIES_1_023_CA_1_validation', 'HOBBIES_1_024_CA_1_validation',
       'HOBBIES_1_025_CA_1_validation', 'HOBBIES_1_026_CA_1_validation',
       'HOBBIES_1_027_CA_1_validation', 'HOBBIES_1_

# 1. Defining Model Functions

In [84]:
def perform_prophet(data_train, data_test):

    temp_rmsse = []

    data_train.reset_index(inplace=True,names="date")
    data_test.reset_index(inplace=True,names="date")
    
    prophet_data_train = data_train[["id","date","sales"]]
    prophet_data_test = data_test[["id","date","sales"]]
    prophet_data_train.columns = ["id","ds","y"]
    prophet_data_test.columns = ["id","ds","y"]
    prophet_data_train['ds'] = pd.to_datetime(prophet_data_train['ds'])
    prophet_data_test['ds'] = pd.to_datetime(prophet_data_test['ds'])
    
    X_train = prophet_data_train["ds"]
    y_train = prophet_data_train["y"]
    X_test = prophet_data_test["ds"]
    y_test = prophet_data_test["y"]
    
    fbp = Prophet()

    model = fbp.fit(prophet_data_train)
    
    predict_placeholder = fbp.make_future_dataframe(28,freq="D")
    
    # Predict on the test data
    y_pred = fbp.predict(predict_placeholder[-28:])
    

    # Calculate and return the error metric for the current fold
    rmsse = calc_rmsse(y_train, y_test, y_pred["yhat"])
    
    return model, rmsse

In [ ]:
# Function to perform Random Forest modeling and calculate MAE
def perform_random_forest(data_train, data_test):
        
    X_train = data_train.drop(columns="sales")
    y_train = data_train["sales"]
    X_test = data_test.drop(columns="sales")
    y_test = data_test["sales"]

    # Fit Random Forest model on the training data
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Predict on the test data
    predictions = model.predict(X_test)

    # Calculate wRMSSE
    rmsse = calc_rmsse(y_train,y_test, predictions)
    
    return model, rmsse


In [88]:
def perform_auto_arima(data_train,data_test):
    

    y_train = data_train["sales"]
    y_test = data_test["sales"]

    # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
    model = auto_arima(y_train, start_p=0, start_q=0, max_p=5, max_q=5, d=1,
                       seasonal=True, trace=False, error_action='ignore', 
                       suppress_warnings=True, stepwise=True)
    
    # Predict on the test data
    predictions = model.predict(n_periods=len(y_test))

    # Calculate and return the error metric for the current fold
    rmsse = calc_rmsse(y_train,y_test,predictions)
    
    return model, rmsse

In [32]:
def objective_optuna(trial, y_train, y_test):
    
    trend = trial.suggest_categorical('trend', ['add'])
    seasonal = trial.suggest_categorical('seasonal', [None, 'add'])
    seasonal_periods = trial.suggest_categorical('seasonal_periods', [None, 4, 7, 12])
    
    product_results = []

    # Fit Holt-Winters model on the training data
    model = ExponentialSmoothing(y_train, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods,freq='D')
    fitted_model = model.fit(optimized=True)

    # Predict on the test data
    predictions = fitted_model.forecast(steps=len(y_test))

    # Calculate and store the error metric
    mae = mean_absolute_error(y_test, predictions)
    

    # Calculate and return the error metric for the current fold
    rmsse = calc_rmsse(y_train,y_test, predictions)
    product_results.append(rmsse)

    # Average MAE for this product
    average_rmsse = np.mean(product_results)
    return average_rmsse

In [107]:
def perform_exp_smoothing(data_train, data_test):
    y_train = data_train["sales"]
    y_test = data_test["sales"]
    # Create a study object
    study = optuna.create_study(direction='minimize')
    
    print(f"Optimizing hyperparameters for product: {id}")
    
    
    # Run the optimization process for the current product
    study.optimize(lambda trial: objective_optuna(trial, y_train, y_test), n_trials=10, n_jobs=-1)

    # Get the best hyperparameters and the corresponding best MAE
    best_params = study.best_params
    best_rmsse = study.best_value

    # Create the best model with the obtained hyperparameters
    best_model = ExponentialSmoothing(y_train, **best_params).fit()
    
    return best_model, best_rmsse

In [102]:
import lightgbm as lgb

def perform_lightgbm(data_train, data_val, data_test):
    
    X_train = data_train.drop(columns="sales")
    y_train = data_train["sales"]
    X_val = data_val.drop(columns="sales")
    y_val = data_val["sales"]
    X_test = data_test.drop(columns="sales")
    y_test = data_test["sales"]

    
    
    
    # Define LightGBM parameters
    params = {
        "n_estimators": 1000,
        'objective': 'regression',
        'metric': 'rmse',
        "boosting_type": "gbdt",
        "max_depth": -1,
        'learning_rate': 0.01,
        'feature_fraction': 0.4,
        "lambda_l1": 1,
        "lambda_l2": 1,
        "seed": 46,
        "verbose": -1
    }

    model = lgb.LGBMRegressor(**params)
    
    # Create dataset for LightGBM
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    
    # Train the model
    num_round = 1000

    bst = lgb.train(params, lgb_train, num_round, valid_sets=lgb_eval, callbacks=[lgb.early_stopping(stopping_rounds=50)])
     
    # Make predictions for the next 28 days
    predictions = bst.predict(X_test, num_iteration=bst.best_iteration)

    # Calculate and return the error metric for the current fold
    rmsse = calc_rmsse(y_train,y_test, predictions)
    
    return bst, rmsse


# 2.Running all models in a loop to find for each product with lowest score

In [110]:
models_list = ["RandomForest","LightGBM","ExponentialSmoothing","ARIMA","Prophet"]

In [ ]:
from pmdarima import auto_arima

# Dictionary to store MAE results for each unique time-series identified by id
product_results = {}
average_rmsse = []

ids_filtered=merge_df_scaled['id'].unique()
filtered_df = merge_df_scaled[merge_df_scaled['id'].isin(ids_filtered)]
weights=calculate_last_28_days_weights(filtered_df)   

# Iterate over each unique product series identified by id
for id in ids_filtered:
    print(f"Analyzing product: {id}")
    product_data = merge_df_scaled[merge_df_scaled['id'] == id].drop(columns="id")
    product_data_with_id = merge_df_scaled[merge_df_scaled['id'] == id]

    # Results list for the current product time-series
    results = {}
    best_score = 999.99
    best_model_name = ""


    product_weight=weights.loc[id].weights



    #Looping all models
    for model_name in models_list:

        if model_name == "ARIMA":
            rmsse_list = []
            # Fit ARIMA model on the training data using auto_arima to find the best (p, d, q)
            for splits in custom_ttsplit(product_data):
                data_train, data_test = splits[0], splits[1]
                model, rmsse = perform_auto_arima(data_train,data_test)
                rmsse_list.append(rmsse)

            rmsse = np.mean(rmsse_list)
            results[model_name] = {"rmsse": rmsse, "model": model}
            if rmsse < best_score:
                best_score = rmsse
                best_model = model
                best_model_name = model_name

        elif model_name == "ExponentialSmoothing":
            rmsse_list = []
            for splits in custom_ttsplit(product_data):
                data_train, data_test = splits[0], splits[1]
                model, rmsse = perform_exp_smoothing(data_train, data_test)
                rmsse_list.append(rmsse)

            rmsse = np.mean(rmsse_list)
            results[model_name] = {"rmsse": rmsse, "model": model}
            if rmsse < best_score:
                best_score = rmsse
                best_model = model
                best_model_name = model_name

        elif model_name == "Prophet":
            rmsse_list = []
            for splits in custom_ttsplit(product_data_with_id):
                data_train, data_test = splits[0], splits[1]
                model, rmsse = perform_prophet(data_train,data_test)
                rmsse_list.append(rmsse)

            rmsse = np.mean(rmsse_list)
            results[model_name] = {"rmsse": rmsse, "model": model}
            if rmsse < best_score:
                best_score = rmsse
                best_model = model
                best_model_name = model_name


        elif model_name == "LightGBM":
            rmsse_list = []

            for splits in custom_ttsplit_inclval(product_data):
                data_train, data_val, data_test = splits[0], splits[1], splits[2]

                model, rmsse = perform_lightgbm(data_train, data_val, data_test)
                rmsse_list.append(rmsse)

            rmsse = np.mean(rmsse_list)
            results[model_name] = {"rmsse": rmsse, "model": model}
            if rmsse < best_score:
                best_score = rmsse
                best_model = model
                best_model_name = model_name


        elif model_name == "RandomForest":
            rmsse_list = []

            for splits in custom_ttsplit_onlyfeatures(product_data):
                data_train, data_test = splits[0], splits[1]

                model, rmsse = perform_random_forest(data_train, data_test)
                rmsse_list.append(rmsse)

            rmsse = np.mean(rmsse_list)
            results[model_name] = {"rmsse": rmsse, "model": model}
            if rmsse < best_score:
                best_score = rmsse
                best_model = model
                best_model_name = model_name


    #Printing results for this product
    print(results)
    print(f"Model results for {id}")
    print(f"Best model: {best_model_name}")
    print(f"Best score: {best_score}")

    average_rmsse.append(best_score*product_weight)

    # Store the average MAE for the current product time-series
    product_results[id] = {"best_score": best_score, "best_model": best_model_name, "model": best_model}

    #Store the best model in a pkl file
    filename = f'../models/{id}_model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(best_model, f)

# Create a DataFrame to store the results
results_df_arima = pd.DataFrame(product_results.items(), columns=['id', 'RMSSE'])

# Set the 'id' column as the index
results_df_arima.set_index('id', inplace=True)

average_rmsse_score = np.sum(average_rmsse)

print(f"Total average wRMSSE: {average_rmsse_score}")


/tmp/ipykernel_4342/714081597.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_28_days_df['revenues'] = last_28_days_df['sales'] * last_28_days_df['sell_price']


Analyzing product: HOBBIES_1_001_CA_1_validation
X_train:
            wday  month  year  snap_CA  snap_TX  snap_WI  sell_price  \
date                                                                   
2011-01-29     1      1   0.0        0        0        0    0.000000   
2011-01-30     2      1   0.0        0        0        0    0.000000   
2011-01-31     3      1   0.0        0        0        0    0.000000   
2011-02-01     4      2   0.0        1        1        0    0.000000   
2011-02-02     5      2   0.0        1        0        1    0.000000   
...          ...    ...   ...      ...      ...      ...         ...   
2016-03-23     5      3   1.0        0        0        0    0.266624   
2016-03-24     6      3   1.0        0        0        0    0.266624   
2016-03-25     7      3   1.0        0        0        0    0.266624   
2016-03-26     1      3   1.0        0        0        0    0.266624   
2016-03-27     2      3   1.0        0        0        0    0.266624   

     

/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[885]	valid_0's rmse: 0.816008
Training until validation scores don't improve for 50 rounds


/home/jonas/.pyenv/versions/3.10.6/envs/walmart/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[I 2024-05-13 18:02:17,360] Trial 0 finished with value: 0.8486981149874614 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 7}. Best is trial 0 with value: 0.8486981149874614.
[I 2024-05-13 18:02:17,636] Trial 9 finished with value: 0.8486981149874614 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': None}. Best is trial 0 with value: 0.8486981149874614.
[I 2024-05-13 18:02:17,646] Trial 4 finished with value: 0.8486981149874614 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': None}. Best is trial 0 with value: 0.8486981149874614.
[I 2024-05-13 18:02:17,695] Trial 6 finished with value: 0.8486981149874614 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 7}. Best is trial 0 with value: 0.8486981149874614.
[I 2024-05-13 18:02:17,726] Trial 7 finished with value: 0.8486981149874614 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 7}. Best is trial 0 with value: 0.8486981149874614.

Optimizing hyperparameters for product: HOBBIES_1_005_CA_1_validation


[I 2024-05-13 18:02:19,744] Trial 1 finished with value: 0.6770543859981519 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': None}. Best is trial 1 with value: 0.6770543859981519.
[I 2024-05-13 18:02:19,752] Trial 6 finished with value: 0.6770543859981519 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 12}. Best is trial 1 with value: 0.6770543859981519.
[I 2024-05-13 18:02:19,763] Trial 2 finished with value: 0.6770543859981519 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 12}. Best is trial 1 with value: 0.6770543859981519.
[I 2024-05-13 18:02:19,766] Trial 5 finished with value: 0.6770543859981519 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 12}. Best is trial 1 with value: 0.6770543859981519.
[I 2024-05-13 18:02:19,811] Trial 7 finished with value: 0.6770543859981519 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 12}. Best is trial 1 with value: 0.6770543859981519

Optimizing hyperparameters for product: HOBBIES_1_005_CA_1_validation


[I 2024-05-13 18:02:21,164] Trial 0 finished with value: 0.9259697602506952 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': None}. Best is trial 0 with value: 0.9259697602506952.
[I 2024-05-13 18:02:21,854] Trial 2 finished with value: 0.9259697602506952 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 12}. Best is trial 0 with value: 0.9259697602506952.
[I 2024-05-13 18:02:22,309] Trial 6 finished with value: 0.9259697602506952 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 4}. Best is trial 0 with value: 0.9259697602506952.
[I 2024-05-13 18:02:22,420] Trial 8 finished with value: 0.9259697602506952 and parameters: {'trend': 'add', 'seasonal': None, 'seasonal_periods': 4}. Best is trial 0 with value: 0.9259697602506952.
[I 2024-05-13 18:02:22,865] Trial 5 finished with value: 0.9307495086037341 and parameters: {'trend': 'add', 'seasonal': 'add', 'seasonal_periods': 4}. Best is trial 0 with value: 0.9259697602506952.
